In [1]:
import pandas as pd
import zipfile

In [2]:
# https://stackoverflow.com/questions/40744027/using-pandas-read-csv-with-zip-compression/45771620#45771620
zf = zipfile.ZipFile("archive.zip") 
df_application = pd.read_csv(zf.open("application_record.csv"))
df_credit = pd.read_csv(zf.open("credit_record.csv"))

In [3]:
df_application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [4]:
df_credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [12]:
# Put df_credit into a dictionary
# dict_credit's keys will be the IDs,
# The value will be another dictionary with months_balance as 
# the key and status as the value
dict_credit = {}
for index in df_credit.index:
    if (df_credit.at[index, "ID"] not in dict_credit):
        dict_credit[df_credit.at[index, "ID"]] = {df_credit.at[index, "MONTHS_BALANCE"]:df_credit.at[index, "STATUS"]}
    else:
        dict_credit[df_credit.at[index, "ID"]][df_credit.at[index, "MONTHS_BALANCE"]] = df_credit.at[index, "STATUS"]
        


In [40]:
# Now transform the dictionary into an array that's like
# [last month's status, ..., first month's status]
# A person can have record from -50 month to -36 month and stopped
# using the service.
for _, (ID, value) in enumerate(dict_credit.items()):
    a_list = []
    len_months = -len(value)
    
    # Set a while loop to change value -> list
    i = 0
    start = 0
    while (len_months != i):
        if (i not in value):
            len_months -= 1
        else:
            start = 1
            a_list.append(value[i])
        i -= 1
        
    dict_credit[ID] = a_list

In [41]:
# Simple way to turn the information into a value
# is to count how long this person has been using
# our service

dict_months = dict_credit.copy()

for _, (ID, records) in enumerate(dict_months.items()):
    dict_months[ID] = len(records)